In [1]:
alpha_name = "AlphaNetV1_Shift_1_Sequence_20_10d_ret_standscalar"
start_date = '2019-01-01'
end_date = '2021-06-01'
change = 5

In [2]:
import time
from Research.backtest.bt import BTDaily
from Research.feature.ft import FeatureAnalysis
import Research.utils.namespace as namespace
import Research.utils.normalization as norm
import numpy as np
import pandas as pd
import DataAPI
import os
input_path = r'/home/wuwenjun/factor_lib/ti0/wuwenjun/' + alpha_name
output_path = r'/home/wuwenjun/factor_lib/ti0/wuwenjun/'

alpha_list = []
trading_list = DataAPI.get_trading_days(start_date, end_date)
count = 0
for day in trading_list:
    file = os.path.join(input_path, str(day.year), day.strftime('%Y%m%d') + '.csv')
    try:
        df = pd.read_csv(file, dtype={0: str})
    except FileNotFoundError:
        count += 1
        continue
    flag = count % change
    path_output = os.path.join(output_path, str(alpha_name + "_" + str(flag)), str(day.year), day.strftime('%Y%m%d') + '.csv')
    alpha_list.append(str(alpha_name + "_" + str(flag)))
    if not os.path.exists(path_output):
        os.makedirs(os.path.dirname(path_output), exist_ok=True)
    df.to_csv(path_output, index=False, header=False)
    count += 1

In [3]:
pd.set_option('expand_frame_repr', False)
config_path = r'/home/ShareFolder/lgc/Modules/Research/config/feature_bt_template'
print('Loading the configuration from ' + config_path)
configs = namespace.load_namespace(config_path)
FT = FeatureAnalysis(configs, feature_path=r"/home/wuwenjun/factor_lib")
alpha_list = list(set(alpha_list))
print(alpha_list)
FT.load_feature_from_file(alpha_list, start_date, end_date, universe='Investable',
                                       timedelta=None, transformer=norm.standard_scale)
FT.load_return_data()

writing_path = r'/home/wuwenjun/Desktop/%s.txt' % alpha_name
f = open (writing_path,'w')
print("\n",file=f)
f.close()
for alpha_name in alpha_list:
    FT.get_intersection_ic(feature_name=alpha_name, truncate_fold=None, method='spearman',
                                     period=('1d', '3d', '5d'))
    ic_flag, trading_direction = FT.test_ic(alpha_name, verbose=False)
    df, df_all = FT.get_ic_summary_by_month(num=6)
    
    if trading_direction == -1:
        negative = True
    else:
        negative = False
    f = open (writing_path,'a')
    print("*"*100,end="\n"*3,file=f)
    print(alpha_name,end = "\n"*3,file=f)
    print(df,end="\n"*3,file = f)
    a = FT.get_top_return(alpha_name, negative= False, trade_type='long-only', transmission_rate=0.00025,
                            tax_rate=0.001, verbose=True,bt_price = "vwap",trade_period=6)
    print(a[1],end = "\n"*3+"*"*100 ,file = f)
    f.close()

Loading the configuration from /home/ShareFolder/lgc/Modules/Research/config/feature_bt_template


Loading Feature...:   3%|██▍                                                                                       | 78/2920 [00:00<00:03, 748.04it/s]

['AlphaNetV1_Shift_1_Sequence_20_10d_ret_standscalar_4', 'AlphaNetV1_Shift_1_Sequence_20_10d_ret_standscalar_1', 'AlphaNetV1_Shift_1_Sequence_20_10d_ret_standscalar_2', 'AlphaNetV1_Shift_1_Sequence_20_10d_ret_standscalar_3', 'AlphaNetV1_Shift_1_Sequence_20_10d_ret_standscalar_0']


Calculating IC value...: 100%|█████████████████████████████████████████████████████████████████████████████████████| 244/244 [00:00<00:00, 722.62it/s]



IC table for feature "AlphaNetV1_Shift_1_Sequence_20_10d_ret_standscalar_4" (mean std IR):
+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|      period       |  IC_AlphaNetV1_1d  |  IC_AlphaNetV1_2d  |  IC_AlphaNetV1_3d  |  IC_AlphaNetV1_4d  |  IC_AlphaNetV1_5d  |
+===================+====================+====================+====================+====================+====================+
| 20190108-20190630 |   2.52 4.28 0.59   |   3.51 5.11 0.69   |   3.94 5.06 0.78   |   5.49 4.70 1.17   |   6.05 4.54 1.33   |
+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
| 20190701-20191231 |   1.46 4.83 0.30   |   1.86 4.11 0.45   |   2.82 4.72 0.60   |   3.63 4.45 0.82   |   3.53 4.21 0.84   |
+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+

IC summary for fea

...

Traceback (most recent call last):
  File "/usr/local/python3/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/local/python3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/local/python3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):


TypeError: object of type 'NoneType' has no len()

Traceback (most recent call last):
  File "/usr/local/python3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/python3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/python3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/local/python3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/usr/local/python3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/python3/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter_

  File "/usr/local/python3/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/usr/local/python3/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/usr/local/python3/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/usr/local/python3/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/usr/local/python3/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/usr/local/python3/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/usr/local/python3/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/usr/local/python3/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/usr/local/python3/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/usr/local/python3/lib/python3.7/multiprocessing

Process ForkPoolWorker-97:
Process ForkPoolWorker-99:
Process ForkPoolWorker-101:
Process ForkPoolWorker-96:
Process ForkPoolWorker-100:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/python3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/python3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/python3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/local/python3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/python3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lo